<a href="https://colab.research.google.com/github/runawaycow/TAP_Project/blob/main/Image_Ranking_Paper_Survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ChatGPT4.5 Prompt:**
Please write a python code for the following task: I will give you a .csv file, the file. Each row include a matching result among 6 players (randomly selected in the pool of 40-50 player). The .csv include following column 'Rank_1 , Rank_2, .... Rank_6' contains the ID of player ranked in each of the match, where Rank_6 has the player ID of the highest place, and Rank_1 contains the player ID in lowest place. The 'SheetID' collunm contains the match ID of the game played. Please loop through all matches and rank the pool of players accordingly between 0-1 with Trueskill, where weakest player is closer to 0 and stronger is closer to 1.

Please also save the ranking result to 'ranking_total.csv', sorted by player ID (stored in a collunm named 'ImageID') , the ranking value of each player is stored in the matching position under collunm 'PreferenceRanking'.

# **Ensure you have the TrueSkill library installed:**

In [1]:
! pip install trueskill

  Preparing metadata (setup.py) ... done
  Created wheel for trueskill: filename=trueskill-0.4.5-py3-none-any.whl size=18049 sha256=190e7e478e8432c83de7106dce55bc40bdb9577c35aaf1e762431c03ff125fef
  Stored in directory: /root/.cache/pip/wheels/b1/20/f2/bb1bca2728bad41aa94d5bf7ca2f8148fd00d2c2e3adfe2ea2
Successfully built trueskill


# **Instructions to run the code:**

1.   Replace 'your_file.csv' with the path to your CSV file.
2.   Ensure your CSV file columns are exactly named: SheetID, Rank_1, Rank_2,Rank_3, Rank_4, Rank_5, Rank_6.
3.  Run the Python script to get player rankings normalized between 0 (weakest) and 1 (strongest) based on TrueSkill ratings.

This will print out the normalized rankings from 0 (weakest) to 1 (strongest) for each player.

In [2]:
import pandas as pd
import trueskill

# Load CSV file
df = pd.read_csv('/content/survey_info_jules_01_clean.csv')

# Initialize TrueSkill environment
env = trueskill.TrueSkill()
ratings = {}

# Loop through each match and update TrueSkill ratings
for _, row in df.iterrows():
    match_results = []
    for rank in range(6, 0, -1):  # Start from Rank_6 (highest) to Rank_1 (lowest)
        player_id = row[f'Rank_{rank}']
        if player_id not in ratings:
            ratings[player_id] = env.create_rating()
        match_results.append([ratings[player_id]])

    updated_ratings = env.rate(match_results)

    # Update player ratings after each match
    for idx, rank in enumerate(range(6, 0, -1)):
        player_id = row[f'Rank_{rank}']
        ratings[player_id] = updated_ratings[idx][0]

# Calculate normalized skill rating (between 0 and 1)
mu_values = {player: rating.mu for player, rating in ratings.items()}
min_mu, max_mu = min(mu_values.values()), max(mu_values.values())

normalized_scores = {player: (mu - min_mu) / (max_mu - min_mu)
                     for player, mu in mu_values.items()}

# Convert to DataFrame for clarity
ranking_df = pd.DataFrame.from_dict(normalized_scores, orient='index', columns=['PreferenceRanking'])
ranking_df.index.name = 'ImageID'
ranking_df.reset_index(inplace=True)
ranking_df.sort_values(by='ImageID', inplace=True)

# Save ranking result to CSV
ranking_df.to_csv('ranking_total.csv', index=False)

# Output ranking
print(ranking_df)


    ImageID  PreferenceRanking
20   750000           0.276698
26   750004           0.349627
23   750014           0.366037
13   750017           0.533217
9    750022           0.353694
6    750030           0.753174
24   750038           0.242669
28   750059           0.305777
40   750068           0.661161
4    750081           0.341018
8    750144           0.527524
25   750157           0.097653
22   750176           0.341545
19   750193           0.355800
37   750216           0.471737
2    750278           0.420116
36   750288           0.211576
16   750376           0.379491
39   750436           0.378380
38   750477           0.547880
10   890021           0.404766
12   890035           0.631716
18   890040           0.456785
11   890051           0.408800
29   890065           0.310126
15   890086           0.564889
30   890100           1.000000
3    890125           0.722160
34   890165           0.564943
33   890182           0.512743
35   890184           0.972720
17   890